In [39]:
import openai
import time
import json
import yfinance as yf

Function that gets the stock price

In [40]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price

Define the function for Open-AI

In [41]:
tools_list = [{
    "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
}]

Initialize open ai

In [42]:
client = openai.OpenAI()

Step 1: Create an Assistant

In [43]:
assistant = client.beta.assistants.create(
    name="Data Analyst Assistant",
    instructions="You are a personal Data Analyst",
    model="gpt-4-1106-preview",
    tools=tools_list
)

Step 2: Create a Thread

In [44]:
thread = client.beta.threads.create()

Step 3: Add a Message to a Thread

In [45]:
messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you please provide me stock price of Apple?"
)

Step 4: Run the Assistant

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Raul."
)

print(run.model_dump_json(indent=4))

# Get the status of the run
while True:
    # Wait for 5 seconds
    time.sleep(5)

    # Retrieve run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run_status.model_dump_json(indent=4))

    # If the run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        # Loop through messages and print content based on the role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")
        break
    elif run_status.status == 'requieres_action':
        print("Requires action")
        print(run_status.model_dump_json(indent=4))
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(5)

{
    "id": "run_N84B9csnCJU09rLPmXNBWvtX",
    "assistant_id": "asst_EKeTURoUOFvE3wUCcqBZYglx",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1699982009,
    "expires_at": 1699982609,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Raul.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-1106-preview",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_jlWDSnpXSZ6t6qUHP9cr07vR",
    "tools": [
        {
            "function": {
                "description": "Retrieve the latest closing price of a stock using its ticker symbol",
                "name": "get_stock_price",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The ticker symbol of the st

{
    "id": "run_N84B9csnCJU09rLPmXNBWvtX",
    "assistant_id": "asst_EKeTURoUOFvE3wUCcqBZYglx",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1699982009,
    "expires_at": 1699982609,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Raul.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-1106-preview",
    "object": "thread.run",
    "required_action": {
        "submit_tool_outputs": {
            "tool_calls": [
                {
                    "id": "call_AS0kwISEEVxOxUUrNGLZPJA6",
                    "function": {
                        "arguments": "{\"symbol\":\"AAPL\"}",
                        "name": "get_stock_price"
                    },
                    "type": "function"
                }
            ]
        },
        "type": "submit_tool_outputs"
    },
    "started_at": 1699982009,
    "status": "requires_action",
    "thread_id": "thread_jlWDSnpXSZ6t6qUHP9cr07vR",
    "tools"